# Install and import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U "transformers>=4.41" bitsandbytes sentencepiece \
                 accelerate datasets evaluate sentence-transformers bert-score rouge_score

In [ ]:
import random, textwrap, torch, evaluate
from tqdm.auto import tqdm
from datasets import Dataset
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Flan-T5-base

In [ ]:
SEED          = 42
MODEL_NAME    = "google/flan-t5-base"
MAX_SRC_TOK   = 256
MAX_NEW_TOK   = 15
BATCH_GEN     = 32
NUM_BEAMS     = 4
K_SHOTS       = 3
DECODE_MODES  = ["greedy", "beam", "topk", "topp"]
PROMPT_MODES  = ["zero", "one", "few"]

random.seed(SEED); torch.manual_seed(SEED)

df   = Dataset.from_csv("/content/drive/MyDrive/wikihow_clean.csv").to_pandas()
data = Dataset.from_pandas(df).shuffle(seed=SEED)

tmp        = data.train_test_split(test_size=0.20, seed=SEED)
train_full = tmp["train"]
temp       = tmp["test"].train_test_split(test_size=0.50, seed=SEED)
val, test  = temp["train"], temp["test"]

# EMBEDDER + pre-calcolo shots
embedder  = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")
train_emb = embedder.encode(train_full["text"], convert_to_tensor=True)

def topk_shots(article, k=K_SHOTS):
    hits = util.semantic_search(embedder.encode(article, convert_to_tensor=True),
                                train_emb, top_k=k)[0]
    return [(train_full[i["corpus_id"]]["text"],
             train_full[i["corpus_id"]]["summary"]) for i in hits]

print("⏳ Pre-computing few-shot cache …")
shots_cache = [topk_shots(t) for t in tqdm(test["text"])]

# PROMPT builder
SYS = "You are an expert WikiHow editor."
      "Summarize this article:"

def build_prompt(article, shots=None):
    blocks = [SYS]
    if shots:
        for a, h in shots:
            blocks.append(f"Article: {a}\nHeadline: {h}")
    blocks.append(f"Article: {article}\nHeadline:")
    return "\n\n".join(blocks)

# MODEL
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(
            MODEL_NAME, torch_dtype=torch.float16, device_map={"":0})
model.eval()

# GENERATION
def generate(idx_list, d_mode, p_mode):
    prompts = []
    for idx in idx_list:
        art = test[idx]["text"]
        if p_mode == "zero":
            shots = None
        elif p_mode == "one":
            shots = shots_cache[idx][:1]
        else:  # few
            shots = shots_cache[idx]
        prompts.append(build_prompt(art, shots))

    enc = tok(prompts, truncation=True, max_length=MAX_SRC_TOK,
              padding=True, return_tensors="pt").to("cuda")

    gen_kwargs = dict(max_length=MAX_NEW_TOK)
    match d_mode:
        case "greedy": gen_kwargs |= {"num_beams":1, "do_sample":False}
        case "beam"  : gen_kwargs |= {"num_beams":NUM_BEAMS, "do_sample":False}
        case "topk"  : gen_kwargs |= {"do_sample":True, "top_k":50,
                                      "temperature":0.7, "num_beams":1}
        case "topp"  : gen_kwargs |= {"do_sample":True, "top_p":0.9,
                                      "temperature":0.7, "num_beams":1}

    with torch.no_grad():
        outs = model.generate(**enc, **gen_kwargs)

    return [tok.decode(o, skip_special_tokens=True).strip() for o in outs]

# METRICS
rouge  = evaluate.load("rouge")
berts  = evaluate.load("bertscore")
results = {}

for p_mode in PROMPT_MODES:
    preds = {m: [] for m in DECODE_MODES}
    refs  = []
    for i in tqdm(range(0, len(test), BATCH_GEN), desc=f"Prompt={p_mode}"):
        idx_batch = list(range(i, min(i+BATCH_GEN, len(test))))
        refs.extend(test[idx_batch]["summary"])
        for d_mode in DECODE_MODES:
            preds[d_mode].extend(generate(idx_batch, d_mode, p_mode))


    res = {}
    for d_mode in DECODE_MODES:
        r = rouge.compute(predictions=preds[d_mode], references=refs)
        b = berts.compute(predictions=preds[d_mode], references=refs, lang="en")
        res[d_mode] = {
            "R1": r["rouge1"], "R2": r["rouge2"],
            "RL": r["rougeL"], "BF1": sum(b["f1"])/len(b["f1"])
        }
    results[p_mode] = res

for p_mode, score in results.items():
    print(f"\n=== {p_mode.upper()}-SHOT ===")
    for d_mode, v in score.items():
        print(f"{d_mode:<5}  R1 {v['R1']:.4f}  R2 {v['R2']:.4f}  "
              f"RL {v['RL']:.4f}  B-F1 {v['BF1']:.4f}")

print("\n=== 3 EXAMPLES ===")
for idx in random.sample(range(len(test)), 3):
    art  = textwrap.shorten(test[idx]["text"], width=220, placeholder=" …")
    gold = test[idx]["summary"]
    print(f"\n📄 Article: {art}\n✅ Gold   : {gold}")
    for p_mode in PROMPT_MODES:
        for d_mode in DECODE_MODES:
            pred = generate([idx], d_mode, p_mode)[0]
            print(f"🤖 {p_mode[:3]} | {d_mode:<5}: {pred}")


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt=one:   0%|          | 0/396 [00:00<?, ?it/s]

Prompt=few:   0%|          | 0/396 [00:00<?, ?it/s]


=== ZERO-SHOT ===
greedy  R1 0.1123  R2 0.0447  RL 0.1043  B-F1 0.8322
beam   R1 0.0487  R2 0.0162  RL 0.0460  B-F1 0.8222
topk   R1 0.1609  R2 0.0564  RL 0.1505  B-F1 0.8421
topp   R1 0.1688  R2 0.0640  RL 0.1581  B-F1 0.8425

=== ONE-SHOT ===
greedy  R1 0.3663  R2 0.1839  RL 0.3498  B-F1 0.8832
beam   R1 0.3640  R2 0.1942  RL 0.3477  B-F1 0.8816
topk   R1 0.2780  R2 0.1204  RL 0.2649  B-F1 0.8713
topp   R1 0.3030  R2 0.1369  RL 0.2886  B-F1 0.8755

=== FEW-SHOT ===
greedy  R1 0.3656  R2 0.1876  RL 0.3517  B-F1 0.8836
beam   R1 0.3738  R2 0.2029  RL 0.3596  B-F1 0.8839
topk   R1 0.2828  R2 0.1257  RL 0.2712  B-F1 0.8723
topp   R1 0.3065  R2 0.1443  RL 0.2938  B-F1 0.8757

=== 3 EXAMPLES ===

📄 Article: Read this article if you're interested in learning how to create an APA style title page via Google Drive. This article assumes that you already have a Google account .
✅ Gold   : Create an APA Style Title Page via Google Drive
🤖 zer | greedy: Open Google Drive. Click the "Title Page" 